In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import pandas as pd
from scipy import stats
# R-like interface, alternatively you can import statsmodels as import statsmodels.api as sm
import statsmodels.formula.api as smf 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import theano

from scipy.special import logsumexp

%config InlineBackend.figure_formats = ['retina']
plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])

/home/rer/anaconda3/envs/pymc3_resources/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [28]:
p1=np.array([0.2, 0.2, 0.2, 0.2, 0.2])
p2=np.array([0.8, 0.1, 0.05, 0.025, 0.025])
p3=np.array([0.05, 0.15, 0.7, 0.05, 0.05])

Question 1

In [39]:
p = [p1, p2, p3]
H = []
for i in p:
    H.append(-sum(i*np.log(i)))
print(H)

[1.6094379124341005, 0.7430039367341686, 0.9836002995230935]


The largest entropy is found on Island 1, where there is an equal probability of sampling either of the 5 types of birbs.  As there is a particular type of birb that is most common (0.8 probability of sampling on island 2, 0,7 on island 3), then the entropy on these islands is lower.  Island 2 has both the highest prob and the lowest prob of sampling a unique type of birb, so this has the lowest entropy of any island.

The Kullback-Liebler divergence is the cross-entropy - the target entropy H(p,q) - H(p), where p is the target distribution and q is the model distribution.  H(p,q) = -sum(p * log(q)), so K-L Divergence is:

D_kl(p,q) = sum(p * log(p/q))

In [61]:
def kl_div( p, q ):
   "Kullback-Liebler Divergence of target distribution p and model distribution q"
   return sum(p*np.log(p/q))

In [55]:
kl_div(p1,p2)

0.9704060527839233

In [56]:
kl_div(p1,p3)

0.638760437463217

In [57]:
kl_div(p2,p1)

0.8664339756999315

In [58]:
kl_div(p2,p3)

2.010914241472249

In [59]:
kl_div(p3,p1)

0.6258376129110066

In [60]:
kl_div(p3,p2)

1.8388451788909108

As expected, the model distribution p1 has the highest entropy, but provides the lowest KL divergence for target distributions p2 and p3.  The measure is not symmetric, so the measure for target p1 and model p2 and p3 have higher KL divergence.  Island 1 has the largest entropy and therefore using it as a model leads to less 'surprise' when we find a very different distribution, such as on islands 2 and 3.  This contrasts when using island 2 or 3 as the model to predict the others.  Island 2 suggests that we should expect birb type 1 to dominate the island, however the other types are at least as common on island 1 and 3.

Question 2

Need to do in R because data comes from McElreath's (agent-based) simulation on happiness, marriage and age.

Model m6.9, which explains a person's happiness controlling for both age and marriage status has the lowest WAIC and LOO compared to model m6.10 (values 2714 for m6.9 versus 3102 for m6.10), which only controls for age, and not for marriage status.  This suggests that model m6.9 would make better predictions.  However, as discussed in Chapter 6, model m6.10 provides the correct causal inference about the influence of age on happiness (this is known because the simulation is known to us).  Marriage is a collider, so when we condition on it in m6.9, it opens a backdoor path between age and happiness.  It should be expected that model m6.9 would make better predictions because it has additional parameters (to control for the marriage status) and can (over)fit the model more.  Indeed the effective parameter measures 'pWAIC' and 'pLOO' indicate the additional parameter.

From the solutions:
    "The model that produces the invalid inference, m6.9 , is expected to predict
much better. And it would. This is because the collider path does convey
actual association. We simply end up mistaken about the causal inference.
We should not use WAIC (or LOO) to choose among models, unless we have
some clear sense of the causal model. These criteria will happily favor confounded models."